In [6]:
!pip list

Package                           Version
--------------------------------- ------------------
aext-assistant                    4.40.0
aext-assistant-server             4.40.0
aext-core                         4.40.0
aext-core-server                  4.40.0
aext_environments_server          4.40.0
aext-notebook-load-server         4.40.0
aext-panels                       4.40.0
aext-panels-server                4.40.0
aext-project-filebrowser-server   4.40.0
aext-share-notebook-server        4.40.0
aext-shared                       4.40.0
aext-toolbox                      4.40.0
aiobotocore                       2.12.3
aiohttp                           3.9.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.17.1
altair                            5.0.1
anaconda-anon-usage               0.7.4
anaconda-auth                     0.10.0
anaconda-catalogs                 0.2.0
anaconda-

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os, pickle

PDF_PATH = "BOOK.pdf"
SAVE_DIR = "../faiss_layer1"

os.makedirs(SAVE_DIR, exist_ok=True)

# Load PDF
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

# Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)
chunks = text_splitter.split_documents(documents)

# Embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Build FAISS
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index
vectorstore.save_local(SAVE_DIR)

print(f"✅ Layer 1 built with {len(chunks)} chunks")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7b72fe74-b0c7-4f32-88ef-37f62b85296a)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


✅ Layer 1 built with 23 chunks


In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
import os, pickle, faiss
import numpy as np

LAYER1_DIR = "../faiss_layer1"
LAYER2_DIR = "../faiss_layer2"
os.makedirs(LAYER2_DIR, exist_ok=True)

# Load Layer 1
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local(
    LAYER1_DIR,
    embedding_model,
    allow_dangerous_deserialization=True
)

raw_docstore = vectorstore.docstore._dict

# LLM
llm = Ollama(model="gemma3:4b")

summary_texts = []

print(f"Total chunks: {len(raw_docstore)}\n")

for i, (doc_id, doc) in enumerate(raw_docstore.items(), start=1):
    try:
        summary = llm.invoke(
            f"Summarize this text factually without adding new information:\n{doc.page_content}"
        )
        summary_texts.append({
            "id": doc_id,
            "text": summary
        })
        print(f"✅ {i} summarized")
    except Exception as e:
        print(f"❌ {i} failed: {e}")

# Embed summaries
summary_vectors = embedding_model.embed_documents(
    [s["text"] for s in summary_texts]
)

dimension = len(summary_vectors[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(summary_vectors).astype("float32"))

# Save metadata
docstore = {s["id"]: s["text"] for s in summary_texts}
index_to_docstore_id = {i: s["id"] for i, s in enumerate(summary_texts)}

with open(f"{LAYER2_DIR}/index.pkl", "wb") as f:
    pickle.dump(
        {"docstore": docstore, "mapping": index_to_docstore_id},
        f
    )

faiss.write_index(index, f"{LAYER2_DIR}/index.faiss")

print("\n🎉 Layer 2 built successfully")


Total chunks: 23

✅ 1 summarized
✅ 2 summarized
✅ 3 summarized
✅ 4 summarized
✅ 5 summarized
✅ 6 summarized
✅ 7 summarized
✅ 8 summarized
✅ 9 summarized
✅ 10 summarized
✅ 11 summarized
✅ 12 summarized
✅ 13 summarized
✅ 14 summarized
✅ 15 summarized
✅ 16 summarized
✅ 17 summarized
✅ 18 summarized
✅ 19 summarized
✅ 20 summarized
✅ 21 summarized
✅ 22 summarized
✅ 23 summarized

🎉 Layer 2 built successfully


In [16]:
import pickle, faiss
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

# Load embeddings + LLM
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
llm = Ollama(model="gemma3:4b")


BASE_DIR = os.path.dirname(os.path.abspath(r"C:/Users/LS/RAG based on PDF/faiss_layer2"))

FAISS_INDEX_PATH = os.path.join(
    BASE_DIR, "faiss_layer2", "index.faiss"
)

FAISS_META_PATH = os.path.join(
    BASE_DIR, "faiss_layer2", "index.pkl"
)

index = faiss.read_index(FAISS_INDEX_PATH)

with open(FAISS_META_PATH, "rb") as f:
    data = pickle.load(f)


docstore = data["docstore"]
mapping = data["mapping"]

def ask(question, top_k=3):
    q_vec = embedding_model.embed_query(question)
    distances, indices = index.search(
        np.array([q_vec]).astype("float32"),
        top_k
    )

    context = "\n\n".join(
        docstore[mapping[i]] for i in indices[0]
    )

    prompt = f"""
Answer the question ONLY using the context below.
If the answer is not present, say "Not found in the document".

Context:
{context}

Question:
{question}
"""

    return llm.invoke(prompt)

# Example
print(ask("What is this documentation is all about?"))

This tutorial guides users on how to begin using Visual Studio Code. It covers the user interface, customization, code editing features (including IntelliSense and Code Actions), running, debugging, and adding language support via extensions.
